In [ ]:
import requests
import pandas as pd

# Function to get stock data
def get_stock_data(ticker):
    url = f"https://financeapi.net/v11/finance/quoteSummary/{ticker}?modules=financialData,price"
    headers = {"x-api-key": "API_KEY"}  
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        try:
            financial_data = data['quoteSummary']['result'][0]['financialData']
            price_data = data['quoteSummary']['result'][0]['price']
            
            stock_info = {
                "Ticker Name": ticker,
                "Full Name": price_data['longName'],
                "Current Market Price": price_data['regularMarketPrice']['fmt'],
                "Target Mean Price": financial_data['targetMeanPrice']['fmt'],
                "52-week High": financial_data['fiftyTwoWeekHigh']['fmt'],
                "52-week Low": financial_data['fiftyTwoWeekLow']['fmt'],
                # Add trending stocks if needed
            }
            return stock_info
        except KeyError:
            print("Error: Could not find the required data. Check the ticker symbol.")
            return None
    else:
        print(f"Error: API request failed with status code {response.status_code}")
        return None

# function to save data to CSV
def save_to_csv(stock_data, filename="stock_data.csv"):
    df = pd.DataFrame([stock_data])
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

# main flow
ticker = input("Enter the stock ticker symbol: ").upper()
stock_data = get_stock_data(ticker)
if stock_data:
    print(stock_data)
    save_to_csv(stock_data)

In [ ]:
import requests
import matplotlib.pyplot as plt
import datetime

# Function to fetch historical stock data
def get_historical_data(ticker):
    url = f"https://financeapi.net/v8/finance/chart/{ticker}"
    params = {
        "range": "5d",  # Last 5 days
        "interval": "1d"  # Daily interval
    }
    headers = {"x-api-key": "API_KEY"}  
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        try:
            # Extract the relevant data
            timestamps = data['chart']['result'][0]['timestamp']
            highs = data['chart']['result'][0]['indicators']['quote'][0]['high']

            # Convert timestamps to readable dates
            dates = [datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d') for ts in timestamps]

            return dates, highs
        except KeyError:
            print("Error: Could not find historical data. Check the ticker symbol.")
            return None, None
    else:
        print(f"Error: API request failed with status code {response.status_code}")
        return None, None

# function to plot historical data
def plot_historical_highs(ticker, dates, highs):
    plt.figure(figsize=(10, 6))
    plt.plot(dates, highs, marker='o', color='b', label='Highest Price')

    plt.title(f'{ticker} - Highest Stock Prices Over the Last 5 Days')
    plt.xlabel('Date')
    plt.ylabel('Highest Price')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()

    # Save the plot to a file and display
    plt.savefig(f"{ticker}_historical_prices.png")
    plt.show()

# main flow for bonus
ticker = input("Enter the stock ticker symbol for historical data: ").upper()
dates, highs = get_historical_data(ticker)

if dates and highs:
    plot_historical_highs(ticker, dates, highs)